In [8]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [9]:
# coding: utf-8
from __future__ import print_function,division,absolute_import

from traceback import print_exc
import sys
import os
import time
import re

import numpy as np
import tensorflow as tf
import cv2

In [10]:
# 학습당시 모델 설정치들

_model_name = 'vgg_16'
_input_name = 'input'
_image_size = 224
_dataset_num_classes = 2 # len(class_names)
_preprocessing_name = 'vgg_16'
_feature_endpoint = 'vgg_16/fc7'

In [11]:
# 불러올 학습모델

input_checkpoint = 'save/model.ckpt-1000'
#input_checkpoint = './0214/save/model.ckpt-100000'
#input_checkpoint = './22re-vgg16/data/save/model.ckpt-70000'
input_checkpoint

'save/model.ckpt-1000'

In [12]:
# 이미지 파일 읽어오는 함수

import numpy as np
import cv2


def read_image(im_path,flags=cv2.IMREAD_UNCHANGED):
    if im_path.endswith('.npy'):
        im = np.load(im_path)
    else:
        with open(im_path,'rb') as f:
            im_encoded = f.read()
        im = cv2.imdecode(np.frombuffer(im_encoded,dtype=np.uint8),flags=flags)
        if len(im.shape) > 2:
            im = im[:,:,::-1]
    return im

In [13]:
# VDL_InferenceProxy3 경로 설정

#_libpath = os.path.expanduser('C:\\Program Files\\VDLServer\\VDL_InferenceProxy3\\engines\\tf\\lib')
_libpath = os.path.expanduser('../')
_slimpath = os.path.join(_libpath,'slim')
sys.path.insert(0,_slimpath)
sys.path.insert(0,_libpath)

In [14]:
tf.reset_default_graph()

In [15]:
# 네트워크 모델 구성

from nets.nets_factory import get_network_fn


network_fn = get_network_fn(
    _model_name,
    num_classes=_dataset_num_classes,
    is_training=False
)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [16]:
# 텐서플로우 내부 이미지 전처리 구성

from slim.preprocessing.preprocessing_factory import get_preprocessing


preprocessing_fn = get_preprocessing(_preprocessing_name,is_training=False)

image_input = tf.placeholder(name=_input_name,shape=[None,None,3],dtype=tf.uint8)
preprocessed = preprocessing_fn(image_input, _image_size, _image_size)
logits, endpoint = network_fn(tf.expand_dims(preprocessed,axis=0))
logits

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Scale of 0 disables regularizer.


<tf.Tensor 'vgg_16/fc8/squeezed:0' shape=(1, 2) dtype=float32>

In [17]:
# 세션 생성 및 학습모델 로딩

config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess = tf.Session(config=config)
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()
saver.restore(sess, input_checkpoint)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from save/model.ckpt-1000


In [18]:
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants

In [60]:
!pwd

/var/home/joban/Gitrepo/projects/cathod_project_cls


In [58]:
tf.saved_model.utils.build_tensor_info(image_input)

name: "input:0"
dtype: DT_UINT8
tensor_shape {
  dim {
    size: -1
  }
  dim {
    size: -1
  }
  dim {
    size: 3
  }
}

In [59]:
tf.nn.softmax(logits)

<tf.Tensor 'Softmax_3:0' shape=(1, 2) dtype=float32>

In [51]:
tf.saved_model.utils.build_tensor_info(tf.nn.softmax(logits))

name: "Softmax_1:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 1
  }
  dim {
    size: 2
  }
}

In [62]:
!rm -r ./saved/
saved_model_path = './saved/1'
"""Writes SavedModel to disk.
  Args:
    saved_model_path: Path to write SavedModel.
    trained_checkpoint_prefix: path to trained_checkpoint_prefix.
    inputs: The input image tensor to use for detection.
    outputs: A tensor dictionary containing the outputs of a DetectionModel.
  """

saver.restore(sess, input_checkpoint)
builder = tf.saved_model.builder.SavedModelBuilder(saved_model_path)

tensor_info_inputs = {
          'inputs': tf.saved_model.utils.build_tensor_info(image_input)}
tensor_info_outputs = {
    'scores' : tf.saved_model.utils.build_tensor_info(tf.nn.softmax(logits))
}

#for k, v in outputs.items():
#    tensor_info_outputs[k] = tf.saved_model.utils.build_tensor_info(v)
#tensor_info_outputs = tf.saved_model.utils.build_tensor_info(logits)

detection_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
              inputs=tensor_info_inputs,
              outputs=tensor_info_outputs,
        method_name=signature_constants.PREDICT_METHOD_NAME))

builder.add_meta_graph_and_variables(
          sess, [tf.saved_model.tag_constants.SERVING],
          signature_def_map={
              signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                  detection_signature,
          },
      )
# builder.add_meta_graph_and_variables(
#           sess, [tf.saved_model.tag_constants.SERVING],
#           signature_def_map={
#                'predict_images':
#               detection_signature
#           },
#       )

builder.save()



# inp = graph.get_tensor_by_name('import/'+input_name + ':0')
# out = graph.get_tensor_by_name('import/'+output_name + ':0')
    
    
# sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = \
#         tf.saved_model.signature_def_utils.predict_signature_def(
#             {"in": input_tensor}, {"out": output_tensor})

# builder.add_meta_graph_and_variables(sess,
#                                          [tag_constants.SERVING],
#                                          signature_def_map=sigs)

# builder.save()

INFO:tensorflow:Restoring parameters from save/model.ckpt-1000
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved/1/saved_model.pb


b'./saved/1/saved_model.pb'

In [63]:
!cp -r ./saved/ /tmp/my_model/

In [56]:
!saved_model_cli show --dir ./saved/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_UINT8
        shape: (-1, -1, 3)
        name: input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (1, 2)
        name: Softmax_2:0
  Method name is: tensorflow/serving/predict


In [4]:
!saved_model_cli show --dir ~/SERVING_INCEPTION/SERVING_INCEPTION/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict_images']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['images'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: ParseExample/ParseExample:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_STRING
        shape: (-1, 5)
        name: index_to_string_Lookup:0
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 5)
        name: TopKV2:0
  Method name is: tensorflow/serving/predict

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_STRING
        shape: unknown_rank
        name: tf_example:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_STRING
        sh

In [6]:
!saved_model_cli show --dir /tmp/mnist_model/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict_images']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['images'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 784)
        name: x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: y:0
  Method name is: tensorflow/serving/predict

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_STRING
        shape: unknown_rank
        name: tf_example:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_STRING
        shape: (-1, 10)
        name: index_to_string_Lookup:0
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        nam

In [30]:
!saved_model_cli show --dir /tmp/mnist_t/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict_images']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['images'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 784)
        name: x:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: y:0
  Method name is: tensorflow/serving/predict


In [32]:
!saved_model_cli show --dir /tmp/resnet/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['image_bytes'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: input_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: ArgMax:0
    outputs['probabilities'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1001)
        name: softmax_tensor:0
  Method name is: tensorflow/serving/predict

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['image_bytes'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: input_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['classes'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        n

In [19]:
def predict(im_name,preproc_fn=None):
#     im = read_image(im_name)
#     if callable(preproc_fn):
#         im = preproc_fn(im)
#     im = np.tile(im[:,:,np.newaxis],[1,1,3]) # GRAY 에서 RGB 포맷으로 변환
    preds_, = sess.run([logits], {image_input: im_name})
    #print(preds_)
    y = np.argmax(preds_[0])
    score = tf.nn.softmax(preds_[0])[y]
    return y, score, preds_

In [20]:
#from tqdm import tqdm_notebook as tqdm
from imutils import paths

path = 't'
testPaths = sorted(list(paths.list_images(path)))

gt = []
y_pred = []

for path in testPaths:
    im = read_image(path, flags=cv2.IMREAD_COLOR)
    im = cv2.resize(im, (224, 224), interpolation=cv2.INTER_LINEAR)
    y, score, y_pred = predict(im)
    #y_pred.append(y)
    gt.append(path.split('/')[-2])
    print(path, y)

    


t/0/0022_1.jpg 0
t/0/0022_2.jpg 0
t/0/0022_3.jpg 0
t/0/0022_4.jpg 0
t/0/0029_1.jpg 0
t/0/0029_2.jpg 0
t/0/0029_3.jpg 0
t/0/0029_4.jpg 0
t/0/0036_1.jpg 0
t/0/0036_2.jpg 0
t/0/0036_3.jpg 0
t/0/0043_1.jpg 0
t/0/0043_3.jpg 0
t/0/0043_4.jpg 0
t/0/0050_1.jpg 0
t/0/0050_2.jpg 0
t/0/0050_3.jpg 0
t/0/0050_4.jpg 0
t/0/0057_1.jpg 0
t/0/0057_2.jpg 1
t/0/0057_3.jpg 0
t/0/0057_4.jpg 0
t/0/0064_1.jpg 0
t/0/0064_2.jpg 0
t/0/0064_3.jpg 0
t/0/0071_1.jpg 0
t/0/0071_2.jpg 1
t/0/0071_3.jpg 0
t/0/0071_4.jpg 0
t/0/0078_1.jpg 0
t/0/0078_2.jpg 1
t/0/0078_3.jpg 0
t/0/0078_4.jpg 0
t/0/0085_1.jpg 0
t/0/0085_2.jpg 0
t/0/0085_3.jpg 0
t/0/0092_1.jpg 0
t/0/0092_3.jpg 0
t/0/0092_4.jpg 0
t/0/0099_1.jpg 0
t/0/0099_3.jpg 0
t/0/0099_4.jpg 0
t/0/0106_1.jpg 0
t/0/0106_2.jpg 0
t/0/0106_4.jpg 0
t/1/101_48EA.jpg 1
t/1/111_48EA.jpg 1
t/1/113_48EA.jpg 1
t/1/115_48EA.jpg 1
t/1/122_48EA.jpg 1
t/1/134_1EA.jpg 1
t/1/167_48EA.jpg 1
t/1/169_48EA.jpg 1
t/1/171_48EA.jpg 1
t/1/188_48EA.jpg 1
t/1/190_48EA.jpg 1
t/1/197_48EA.jpg 1
t/1/199_

In [65]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

gt = list(map(int, gt))
    
confusion = confusion_matrix(gt, y_pred)
print('confusion matrix:')
print(confusion)
report = classification_report(gt, y_pred, output_dict=False)
print('classification_report:')
print(report)

confusion matrix:
[[37  8]
 [ 0 45]]
classification_report:
              precision    recall  f1-score   support

           0       1.00      0.82      0.90        45
           1       0.85      1.00      0.92        45

   micro avg       0.91      0.91      0.91        90
   macro avg       0.92      0.91      0.91        90
weighted avg       0.92      0.91      0.91        90

